In [61]:
import pandas as pd
import numpy as np
import pathlib as pl
# library used for initial visualization
# library used for initial visualization
import sklearn
import pathlib as pl

In [62]:
path = '../dataset/finRep/FinancialPhraseBank/all-data.csv'

df = pd.read_csv(path, encoding = "ISO-8859-1", names = ['sentiment', 'sentence'])
df

,sentiment,sentence
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [63]:
# path = '../dataset/finRep/SEntFiN-v1.1/SEntFiN-v1.1.csv'

# df = pd.read_csv(path, encoding = "ISO-8859-1")
# df

In [64]:
# import json

# path = '../dataset/finRep/FiQA/task1_headline_ABSA_train.json'

# rows = []
# with open(path) as f:
#     d = json.load(f)

# for key, value in d.items():
#     aux = {'sentence': value['sentence'], 'snippets': value['info'][0]['snippets'], 'target': value['info'][0]['target'], 
#     'sentiment_score': value['info'][0]['sentiment_score'], 'aspects': value['info'][0]['aspects']}
#     rows.append(aux)

# df = pd.DataFrame(rows, columns=['sentence', 'snippets', 'target', 'sentiment_score', 'aspects'])
# df

In [65]:
# !pip install -U spacy

In [80]:
text = df.sample(1)['sentence'].iloc[0]
print(text)

Expense ratio was 102.6 % compared to 92.9 % in the corresponding period in 2005 .


SPACY

In [81]:
import spacy

nlp = spacy.load("en_core_web_md")
doc = nlp(text)
print(doc.ents)

(102.6 %, 92.9 %, 2005)


RAKE NLTK

In [82]:
from rake_nltk import Rake
rake_nltk_var = Rake()

rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()
print(keyword_extracted)

['expense ratio', 'corresponding period', 'compared', '92', '9', '6', '2005', '102']


TFIDF

In [83]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as sw

def train_keyword(data): #SE PUEDE EXTENDER EL DICCIONARIO
#     # use stopword to filter words    
#     stop = sw.stopwords_key()
#     stop = stop.splitlines()
#     stop = str(stop)
#     stop = stop.translate(str.maketrans('', '', string.punctuation))
    stopwords_list = list(sw.words('english'))
#     stopwords_list.extend(stop.split())
    
    # vectorizing the words use tf-idf
    vectorizer = TfidfVectorizer(analyzer='word', stop_words=set(stopwords_list), use_idf=True)
    vector = vectorizer.fit_transform(data)
    
    # store to dataframe    
    df = pd.DataFrame(vector[0].T.todense(), index=vectorizer.get_feature_names(), columns=["vector"])
    df = df.sort_values('vector', ascending=False)
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'keywords'})
    df = df[df['vector'] != 0]

    return df

In [84]:
raw_data = [text]
keywords = train_keyword(raw_data)
# print top 10 keywords
print(keywords.head(10))

        keywords    vector
0            102  0.353553
1           2005  0.353553
2             92  0.353553
3       compared  0.353553
4  corresponding  0.353553
5        expense  0.353553
6         period  0.353553
7          ratio  0.353553


KEYBERT

In [85]:
# # Install the libraries
# !pip install keybert
# !pip install sentence-transformers

In [86]:
# Import them
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

# To show the execution times
# !pip install ipython-autotime
# %load_ext autotime

In [87]:
sent_trans = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
keyBERT_model = KeyBERT(model = sent_trans)

In [88]:
def extract_terms(document, n_gram_range = (3,3), 
                  top_N=5, model=keyBERT_model, diversity_threshold = 0.7):

  keywords = model.extract_keywords(document, stop_words='english', 
                                    keyphrase_ngram_range=(1, 3),
                                    use_mmr=True, 
                                    diversity = diversity_threshold,
                                    top_n = top_N)
  
  return sorted(keywords, key=lambda tup:(-tup[1], tup[0]))

In [89]:
print(f'Text: {text}.\nLength: {len(text.split())}')
best_terms = extract_terms(text)

for terms_and_score in best_terms:
  print(terms_and_score)

Text: Expense ratio was 102.6 % compared to 92.9 % in the corresponding period in 2005 ..
Length: 16
('expense ratio 102', 0.7213)
('92', 0.2755)
('2005', 0.1734)
('compared', 0.1642)
('corresponding period', 0.1249)


In [90]:
# !pip install git+https://github.com/LIAAD/yake

In [91]:
import yake

In [92]:
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(text)

for kw in keywords:
	print(kw)

('Expense ratio', 0.015380821171891606)
('Expense', 0.09568045026443411)
('compared', 0.09568045026443411)
('ratio', 0.15831692877998726)
('period', 0.15831692877998726)


In [93]:
language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)

for kw in keywords:
    print(kw)

('Expense ratio', 0.015380821171891606)
('Expense', 0.09568045026443411)
('compared', 0.09568045026443411)
('ratio', 0.15831692877998726)
('period', 0.15831692877998726)
